In [1]:
import os
import numpy as np
from scipy.io import loadmat
from pyod.models.hbos import HBOS
from pysad.models.integrations import ReferenceWindowModel
from pysad.utils import ArrayStreamer
from pysad.transform.postprocessing import RunningAveragePostprocessor
from pysad.transform.preprocessing import InstanceUnitNormScaler
from pysad.transform.probability_calibration import GaussianTailProbabilityCalibrator
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.ensemble import IsolationForest
import pandas as pd
df = pd.read_csv("Data/Edu_TS_all.csv", sep=";", index_col=0,
                   parse_dates=True)

po = pd.DataFrame(df['Paid Orders'])
iforest = IsolationForest(n_estimators = 100).fit(pd.DataFrame(df['Paid Orders']))

In [3]:
iforest.score_samples(pd.DataFrame(df['Paid Orders']))

array([-0.56566849, -0.59104504, -0.5587706 , -0.5715813 , -0.62563621,
       -0.54065757, -0.72606374, -0.57515761, -0.63275708, -0.57254697,
       -0.61602447, -0.55078508, -0.68228953, -0.54636913, -0.56649658,
       -0.55711294, -0.56160959, -0.57869012, -0.55758042, -0.5637486 ,
       -0.585383  , -0.56649632, -0.50693928, -0.54178917, -0.47096622,
       -0.43533965, -0.44752798, -0.44340751, -0.43015334, -0.50972642,
       -0.45889922, -0.46314828, -0.51146438, -0.43233574, -0.46446488,
       -0.4446895 , -0.45501753, -0.43471378, -0.46304658, -0.44027046,
       -0.47030846, -0.49887798, -0.47784009, -0.485917  , -0.44940029,
       -0.47067283, -0.459941  , -0.42974068, -0.44546754, -0.43912806,
       -0.45325701, -0.46342541, -0.44603172, -0.45896522, -0.56791264,
       -0.54727637, -0.44889493, -0.46064135, -0.464292  , -0.43471378,
       -0.44595213, -0.43912806, -0.47030846, -0.43847883, -0.45896522,
       -0.51900485, -0.45873161, -0.55452364, -0.50771415, -0.47

In [4]:
import numpy as np
from pyod.models.hbos import HBOS
from pysad.models.integrations import ReferenceWindowModel
from pysad.utils import ArrayStreamer
from pysad.transform.postprocessing import RunningAveragePostprocessor
from pysad.transform.preprocessing import InstanceUnitNormScaler
from pysad.transform.probability_calibration import GaussianTailProbabilityCalibrator
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import time
import logging
%matplotlib inline

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', datefmt='%d/%m/%Y %H:%M:%S')

# Function to detect anomalies in real-time
def detect_anomalies(model, new_data):
    # Predict anomaly scores for new data points
    anomaly_scores = model.fit_score_partial(new_data)
    return anomaly_scores

# Example data generation function (replace this with your actual data source)
def generate_data(n_samples=100, n_features=5):
    return np.random.randn(n_samples, n_features)

# Main function
def main():
    # Initialize HBOS model from PyOD
    model_pyod_hbos = HBOS(n_bins=10)

    # Example parameters for window size and sliding size
    window_size = 100
    sliding_size = 30  # seconds

    # Initialize streaming object
    streamer = ArrayStreamer()

    # Initialize ReferenceWindowModel with HBOS model
    model_pysad_refwindow = ReferenceWindowModel(model_cls=model_pyod_hbos, window_size=window_size, sliding_size=sliding_size)

    # Apply preprocessing
    model_pysad_refwindow = InstanceUnitNormScaler(model_pysad_refwindow)

    # Apply postprocessing
    model_pysad_refwindow = RunningAveragePostprocessor(model_pysad_refwindow)

    # Apply probability calibration
    model_pysad_refwindow = GaussianTailProbabilityCalibrator(model_pysad_refwindow)

    # Initialize counter for the number of data points
    data_counter = 0

    # Initialize a variable to control whether the program should continue running
    run_program = True

    while run_program:
        # Generate new data
        logging.info('aboba')
        new_data = generate_data()

        # Stream data into the model
        for data_point in streamer.iter([new_data]):
            data_counter += 1

            # Check if enough data points are streamed to form a window
            if data_counter <= window_size:
                logging.info("Data counter: {}".format(data_counter))
                # Get the data from the streaming object
                window_data = data_point

                # Detect anomalies in the latest window
                anomaly_scores = detect_anomalies(model_pysad_refwindow, window_data)

                logging.info("Anomaly scores:{}".format(anomaly_scores))

            # Sleep until next update time
            logging.info('sleep')
            time.sleep(1)

            # Add condition to break out of the loop (for demonstration, you may need a real condition)
            if data_counter >= 1000:  # for demonstration purposes, you might want to specify a condition to break the loop
                run_program = False
                break

if __name__ == "__main__":
    main()


04/03/2024 01:47:09 - root - INFO - aboba
04/03/2024 01:47:09 - root - INFO - Data counter: 1


AttributeError: 'GaussianTailProbabilityCalibrator' object has no attribute 'fit_score_partial'

In [5]:

X_all = po
iterator = ArrayStreamer(shuffle=False)  # Streamer to simulate streaming data

In [6]:
model = ReferenceWindowModel(model_cls=HBOS, window_size=5, sliding_size=2, initial_window_X=X_all[:10]) # Model initialization
preprocessor = InstanceUnitNormScaler()  # Normalizer
postprocessor = RunningAveragePostprocessor(window_size=100)  # Running average postprocessor
calibrator = GaussianTailProbabilityCalibrator(running_statistics=True, window_size=6)

04/03/2024 01:47:14 - numba.core.byteflow - DEBUG - bytecode dump:
>          0	NOP(arg=None, lineno=270)
           2	RESUME(arg=0, lineno=270)
           4	LOAD_FAST(arg=0, lineno=305)
           6	LOAD_ATTR(arg=0, lineno=305)
          16	LOAD_CONST(arg=1, lineno=305)
          18	BINARY_SUBSCR(arg=None, lineno=305)
          28	LOAD_FAST(arg=0, lineno=305)
          30	LOAD_ATTR(arg=0, lineno=305)
          40	LOAD_CONST(arg=2, lineno=305)
          42	BINARY_SUBSCR(arg=None, lineno=305)
          52	STORE_FAST(arg=7, lineno=305)
          54	STORE_FAST(arg=6, lineno=305)
          56	LOAD_GLOBAL(arg=3, lineno=306)
          68	LOAD_ATTR(arg=2, lineno=306)
          78	LOAD_FAST(arg=6, lineno=306)
          80	LOAD_FAST(arg=7, lineno=306)
          82	BUILD_TUPLE(arg=2, lineno=306)
          84	KW_NAMES(arg=3, lineno=306)
          86	PRECALL(arg=1, lineno=306)
          90	CALL(arg=1, lineno=306)
         100	STORE_FAST(arg=8, lineno=306)
         102	LOAD_GLOBAL(arg=7, lineno=308

In [11]:
iterator.iter(X_all)

<generator object ArrayStreamer.iter at 0x15b37c260>

In [13]:
y_pred = []
for i, X in (iterator.iter(X_all)):
    X = preprocessor.fit_transform_partial(X)
    anomaly_score = model.fit_score_partial(X)
    anomaly_score = postprocessor.fit_transform_partial(anomaly_score)
    calibrated_score = calibrator.fit_transform_partial(anomaly_score)
    y_pred.append(calibrated_score)

fig, ax = plt.subplots(figsize=(8, 5), layout='constrained')
ax.plot(y_pred, label='prediction', linewidth=3)
ax.set_xlabel('time')  # Add an x-label to the axes.
ax.set_ylabel('anomaly likelihood')  # Add a y-label to the axes.
ax.legend()
plt.show()

KeyError: 0

In [ ]:
import numpy as np
from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt

# Define lists to store predicted anomaly scores and ground truth labels
y_pred = []

# Iterate over X_all and y_all starting from index 100
for i, X in tqdm_notebook(enumerate(X_all[100:])):
    # Apply preprocessing to X
    X = preprocessor.fit_transform_partial(X)
    
    # Predict anomaly score for X
    anomaly_score = model.fit_score_partial(X)
    
    # Apply postprocessing to anomaly score
    anomaly_score = postprocessor.fit_transform_partial(anomaly_score)
    
    # Apply probability calibration to anomaly score
    calibrated_score = calibrator.fit_transform_partial(anomaly_score)
    
    # Append calibrated score to y_pred
    y_pred.append(calibrated_score)

# Plotting
fig, ax = plt.subplots(figsize=(8, 5), constrained_layout=True)
ax.plot(y_pred[2000:], label='prediction', linewidth=3)
ax.plot(y_all[2100:], label='ground truth', linewidth=3, linestyle='--')
ax.set_xlabel('time')
ax.set_ylabel('anomaly likelihood')
ax.set_title(f'MNIST dataset with {1e2 * np.count_nonzero(y_all) / len(y_all):.2f}% outliers')
ax.legend()
plt.show()
